In [1]:
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [3]:
n = len(df)
breed = set(df['breed'])
n_class = len(breed)
class_to_num = dict(zip(breed, range(n_class)))
num_to_class = dict(zip(range(n_class), breed))

In [4]:
width = 299
X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, n_class), dtype=np.uint8)
for i in tqdm(range(n)):
    X[i] = cv2.resize(cv2.imread('train/%s.jpg' % df['id'][i]), (width, width))
    y[i][class_to_num[df['breed'][i]]] = 1

100%|███████████████████████████████████| 10222/10222 [00:41<00:00, 249.03it/s]


In [5]:
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [6]:
# define get_features function to extract features using pre-trained CNNs
def get_features(MODEL, data=X):
    cnn_model = MODEL(include_top=False, input_shape=(width, width, 3), weights='imagenet')

    inputs = Input((width, width, 3))
    x = inputs
    x = Lambda(preprocess_input, name='preprocessing')(x)
    x = cnn_model(x)
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs, x)

    features = cnn_model.predict(data, batch_size=64, verbose=1)
    return features

In [7]:
inception_features = get_features(InceptionV3, X)
xception_features = get_features(Xception, X)
resnet_features = get_features(ResNet50, X)
inresv2_features = get_features(InceptionResNetV2, X)

10222/10222 [==============================] - ETA: 1:30:4 - ETA: 1:29:2 - ETA: 1:28:2 - ETA: 1:27:3 - ETA: 1:26:5 - ETA: 1:26:0 - ETA: 1:24:5 - ETA: 1:23:4 - ETA: 1:22:4 - ETA: 1:21:4 - ETA: 1:20:5 - ETA: 1:20:0 - ETA: 1:19:2 - ETA: 1:18:3 - ETA: 1:17:5 - ETA: 1:17:1 - ETA: 1:16:3 - ETA: 1:15:5 - ETA: 1:15:1 - ETA: 1:14:4 - ETA: 1:14:0 - ETA: 1:13:3 - ETA: 1:12:5 - ETA: 1:12:1 - ETA: 1:11:4 - ETA: 1:11:0 - ETA: 1:10:3 - ETA: 1:09:5 - ETA: 1:09:2 - ETA: 1:08:5 - ETA: 1:08:1 - ETA: 1:07:4 - ETA: 1:07:1 - ETA: 1:06:3 - ETA: 1:06:0 - ETA: 1:05:3 - ETA: 1:04:5 - ETA: 1:04:2 - ETA: 1:03:5 - ETA: 1:03:2 - ETA: 1:02:4 - ETA: 1:02:1 - ETA: 1:01:4 - ETA: 1:01:1 - ETA: 1:00:3 - ETA: 1:00:0 - ETA: 59:33  - ETA: 59:0 - ETA: 58:2 - ETA: 57:5 - ETA: 57:2 - ETA: 56:5 - ETA: 56:2 - ETA: 55:4 - ETA: 55:1 - ETA: 54:4 - ETA: 54:1 - ETA: 53:3 - ETA: 53:0 - ETA: 52:3 - ETA: 52:0 - ETA: 51:3 - ETA: 50:5 - ETA: 50:2 - ETA: 49:5 - ETA: 49:2 - ETA: 48:5 - ETA: 48:1 - ETA: 47:4 - ETA: 47:1 - ETA: 46:4 - ETA: 46

In [8]:
# combine all bottleneck features into one for training
features = np.concatenate([inception_features, xception_features, resnet_features, inresv2_features], axis=-1)

In [9]:
# set up our model and train with the bottleneck features from above
# save weights when a better model is produced using the callbacks functionality
inputs = Input(features.shape[1:])
x = inputs
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax')(x)
model = Model(inputs, x)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='weights.best.from_scratch_with_4_predictors.hdf5', verbose=1, save_best_only=True)
h = model.fit(features, y, batch_size=128, epochs=100, callbacks=[checkpointer], validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/100
9199/9199 [==============================] - 5s 514us/step - loss: 1.5189 - acc: 0.6476 - val_loss: 0.5346 - val_acc: 0.8328
Epoch 2/100
9199/9199 [==============================] - 2s 255us/step - loss: 0.4698 - acc: 0.8564 - val_loss: 0.4862 - val_acc: 0.8456
Epoch 3/100
9199/9199 [==============================] - 2s 257us/step - loss: 0.3533 - acc: 0.8902 - val_loss: 0.4558 - val_acc: 0.8543
Epoch 4/100
9199/9199 [==============================] - 2s 254us/step - loss: 0.2798 - acc: 0.9081 - val_loss: 0.4312 - val_acc: 0.8631
Epoch 5/100
9199/9199 [==============================] - 3s 273us/step - loss: 0.2303 - acc: 0.9256 - val_loss: 0.4469 - val_acc: 0.8543
Epoch 6/100


9199/9199 [==============================] - 2s 246us/step - loss: 0.1931 - acc: 0.9387 - val_loss: 0.4207 - val_acc: 0.8602
Epoch 7/100
9199/9199 [==============================] - 2s 246us/step - loss: 0.1662 - acc: 0.9498 - val_loss: 0.4324 - val_acc: 0.8671
Epoch 8/100
9199/9199 [==============================] - 2s 258us/step - loss: 0.1405 - acc: 0.9595 - val_loss: 0.4197 - val_acc: 0.8631
Epoch 9/100
9199/9199 [==============================] - 2s 248us/step - loss: 0.1230 - acc: 0.9623 - val_loss: 0.4146 - val_acc: 0.8602
Epoch 10/100
9199/9199 [==============================] - 2s 261us/step - loss: 0.1114 - acc: 0.9680 - val_loss: 0.4267 - val_acc: 0.8641
Epoch 11/100


9199/9199 [==============================] - 2s 241us/step - loss: 0.0959 - acc: 0.9740 - val_loss: 0.4290 - val_acc: 0.8592
Epoch 12/100
9199/9199 [==============================] - 2s 248us/step - loss: 0.0869 - acc: 0.9780 - val_loss: 0.4262 - val_acc: 0.8671
Epoch 13/100
9199/9199 [==============================] - 2s 250us/step - loss: 0.0787 - acc: 0.9778 - val_loss: 0.4543 - val_acc: 0.8573
Epoch 14/100
9199/9199 [==============================] - 2s 243us/step - loss: 0.0726 - acc: 0.9826 - val_loss: 0.4322 - val_acc: 0.8739
Epoch 15/100
9199/9199 [==============================] - 2s 241us/step - loss: 0.0650 - acc: 0.9838 - val_loss: 0.4399 - val_acc: 0.8680
Epoch 16/100


9199/9199 [==============================] - 2s 241us/step - loss: 0.0606 - acc: 0.9858 - val_loss: 0.4470 - val_acc: 0.8563
Epoch 17/100
9199/9199 [==============================] - 2s 241us/step - loss: 0.0559 - acc: 0.9867 - val_loss: 0.4284 - val_acc: 0.8710
Epoch 18/100
9199/9199 [==============================] - 2s 241us/step - loss: 0.0508 - acc: 0.9897 - val_loss: 0.4304 - val_acc: 0.8573
Epoch 19/100
9199/9199 [==============================] - 2s 250us/step - loss: 0.0474 - acc: 0.9889 - val_loss: 0.4401 - val_acc: 0.8651
Epoch 20/100
9199/9199 [==============================] - 2s 252us/step - loss: 0.0498 - acc: 0.9874 - val_loss: 0.4464 - val_acc: 0.8680
Epoch 21/100


9199/9199 [==============================] - 2s 251us/step - loss: 0.0432 - acc: 0.9911 - val_loss: 0.4545 - val_acc: 0.8700
Epoch 22/100
9199/9199 [==============================] - 2s 243us/step - loss: 0.0408 - acc: 0.9909 - val_loss: 0.4416 - val_acc: 0.8680
Epoch 23/100
9199/9199 [==============================] - 2s 244us/step - loss: 0.0405 - acc: 0.9912 - val_loss: 0.4682 - val_acc: 0.8592
Epoch 24/100
9199/9199 [==============================] - 2s 257us/step - loss: 0.0349 - acc: 0.9929 - val_loss: 0.4613 - val_acc: 0.8690
Epoch 25/100
9199/9199 [==============================] - 2s 248us/step - loss: 0.0350 - acc: 0.9925 - val_loss: 0.4484 - val_acc: 0.8651
Epoch 26/100


...

9199/9199 [==============================] - 2s 234us/step - loss: 0.0198 - acc: 0.9930 - val_loss: 0.6535 - val_acc: 0.8622
Epoch 77/100
9199/9199 [==============================] - 2s 234us/step - loss: 0.0143 - acc: 0.9955 - val_loss: 0.6415 - val_acc: 0.8631
Epoch 78/100
9199/9199 [==============================] - 2s 232us/step - loss: 0.0139 - acc: 0.9962 - val_loss: 0.6280 - val_acc: 0.8553
Epoch 79/100
9199/9199 [==============================] - 2s 232us/step - loss: 0.0147 - acc: 0.9962 - val_loss: 0.6512 - val_acc: 0.8514
Epoch 80/100
9199/9199 [==============================] - 2s 235us/step - loss: 0.0109 - acc: 0.9978 - val_loss: 0.6347 - val_acc: 0.8622
Epoch 81/100


9199/9199 [==============================] - 2s 256us/step - loss: 0.0156 - acc: 0.9954 - val_loss: 0.6492 - val_acc: 0.8602
Epoch 82/100
9199/9199 [==============================] - 2s 236us/step - loss: 0.0142 - acc: 0.9962 - val_loss: 0.6582 - val_acc: 0.8651
Epoch 83/100
9199/9199 [==============================] - 2s 234us/step - loss: 0.0121 - acc: 0.9960 - val_loss: 0.6907 - val_acc: 0.8573
Epoch 84/100
9199/9199 [==============================] - 2s 237us/step - loss: 0.0198 - acc: 0.9941 - val_loss: 0.6908 - val_acc: 0.8524
Epoch 85/100
9199/9199 [==============================] - 2s 234us/step - loss: 0.0132 - acc: 0.9963 - val_loss: 0.6667 - val_acc: 0.8612
Epoch 86/100


9199/9199 [==============================] - 2s 233us/step - loss: 0.0132 - acc: 0.9961 - val_loss: 0.6802 - val_acc: 0.8612
Epoch 87/100
9199/9199 [==============================] - 2s 234us/step - loss: 0.0107 - acc: 0.9973 - val_loss: 0.6838 - val_acc: 0.8612
Epoch 88/100
9199/9199 [==============================] - 2s 235us/step - loss: 0.0124 - acc: 0.9962 - val_loss: 0.6986 - val_acc: 0.8602
Epoch 89/100
9199/9199 [==============================] - 2s 234us/step - loss: 0.0144 - acc: 0.9955 - val_loss: 0.6640 - val_acc: 0.8573
Epoch 90/100
9199/9199 [==============================] - 2s 235us/step - loss: 0.0112 - acc: 0.9964 - val_loss: 0.6736 - val_acc: 0.8534
Epoch 91/100


9199/9199 [==============================] - 2s 235us/step - loss: 0.0159 - acc: 0.9953 - val_loss: 0.7326 - val_acc: 0.8504
Epoch 92/100
9199/9199 [==============================] - 2s 243us/step - loss: 0.0137 - acc: 0.9960 - val_loss: 0.6502 - val_acc: 0.8573
Epoch 93/100
9199/9199 [==============================] - 2s 244us/step - loss: 0.0130 - acc: 0.9968 - val_loss: 0.6776 - val_acc: 0.8651
Epoch 94/100
9199/9199 [==============================] - 2s 238us/step - loss: 0.0136 - acc: 0.9963 - val_loss: 0.6959 - val_acc: 0.8543
Epoch 95/100
9199/9199 [==============================] - 2s 240us/step - loss: 0.0114 - acc: 0.9971 - val_loss: 0.6833 - val_acc: 0.8602
Epoch 96/100


9199/9199 [==============================] - 2s 245us/step - loss: 0.0132 - acc: 0.9963 - val_loss: 0.7244 - val_acc: 0.8504
Epoch 97/100
9199/9199 [==============================] - 2s 259us/step - loss: 0.0140 - acc: 0.9964 - val_loss: 0.6895 - val_acc: 0.8583
Epoch 98/100
9199/9199 [==============================] - 2s 235us/step - loss: 0.0111 - acc: 0.9972 - val_loss: 0.6854 - val_acc: 0.8680
Epoch 99/100
9199/9199 [==============================] - 2s 256us/step - loss: 0.0161 - acc: 0.9942 - val_loss: 0.7476 - val_acc: 0.8612
Epoch 100/100
9199/9199 [==============================] - 2s 254us/step - loss: 0.0182 - acc: 0.9940 - val_loss: 0.7094 - val_acc: 0.8602


In [10]:
# set up to read the image files for the test set and pair them with their correct labels
df2 = pd.read_csv('sample_submission.csv')

n_test = len(df2)
X_test = np.zeros((n_test, width, width, 3), dtype=np.uint8)
for i in tqdm(range(n_test)):
    X_test[i] = cv2.resize(cv2.imread('test/%s.jpg' % df2['id'][i]), (width, width))

100%|███████████████████████████████████| 10357/10357 [00:39<00:00, 261.83it/s]


In [11]:
# using the get_features function to extract bottleneck features from the test set
inception_t_features = get_features(InceptionV3, X_test)
xception_t_features = get_features(Xception, X_test)
resnet_t_features = get_features(ResNet50, X_test)
inresv2_t_features = get_features(InceptionResNetV2, X_test)

10357/10357 [==============================] - ETA: 1:35:1 - ETA: 1:30:1 - ETA: 1:28:0 - ETA: 1:26:3 - ETA: 1:25:3 - ETA: 1:24:5 - ETA: 1:24:1 - ETA: 1:23:3 - ETA: 1:23:0 - ETA: 1:22:2 - ETA: 1:21:4 - ETA: 1:21:0 - ETA: 1:20:3 - ETA: 1:19:5 - ETA: 1:19:1 - ETA: 1:18:4 - ETA: 1:18:0 - ETA: 1:17:3 - ETA: 1:16:5 - ETA: 1:16:2 - ETA: 1:15:4 - ETA: 1:15:1 - ETA: 1:14:3 - ETA: 1:14:0 - ETA: 1:13:3 - ETA: 1:13:0 - ETA: 1:12:3 - ETA: 1:12:0 - ETA: 1:11:3 - ETA: 1:11:0 - ETA: 1:10:3 - ETA: 1:10:0 - ETA: 1:09:3 - ETA: 1:09:0 - ETA: 1:08:2 - ETA: 1:07:5 - ETA: 1:07:2 - ETA: 1:06:4 - ETA: 1:06:1 - ETA: 1:05:4 - ETA: 1:05:0 - ETA: 1:04:3 - ETA: 1:04:0 - ETA: 1:03:3 - ETA: 1:03:0 - ETA: 1:02:2 - ETA: 1:01:5 - ETA: 1:01:2 - ETA: 1:00:4 - ETA: 1:00:1 - ETA: 59:44  - ETA: 59:1 - ETA: 58:4 - ETA: 58:0 - ETA: 57:3 - ETA: 57:0 - ETA: 56:3 - ETA: 55:5 - ETA: 55:2 - ETA: 54:5 - ETA: 54:1 - ETA: 53:4 - ETA: 53:1 - ETA: 52:4 - ETA: 52:1 - ETA: 51:4 - ETA: 51:1 - ETA: 50:4 - ETA: 50:1 - ETA: 49:3 - ETA: 49:0 -

In [12]:
# combine all bottleneck features into one for testing
features_test = np.concatenate([inception_t_features, xception_t_features, resnet_t_features, inresv2_t_features], axis=-1)

In [13]:
# make predictions with our trained model
y_pred = model.predict(features_test, batch_size=128)

In [14]:
# export predictions to a csv file
for b in breed:
    df2[b] = y_pred[:,class_to_num[b]]

df2.to_csv('new_predictions.csv', index=None)